In [2]:
import pandas as pd
from pathlib import Path

In [4]:
df = pd.read_parquet('../data/output/nyc_crashes_ready_for_analysis.parquet')

In [6]:
df.head(30)

,date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,cross_street_name,off_street_name,...,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,is_holiday_bool,global,states,global_across_us,is_holiday
0,2025-01-07,11:51,QUEENS,11357,40.779884,-73.803210,"(40.779884, -73.80321)",160 STREET,WILLETS POINT BOULEVARD,None,...,Sedan,None,None,None,None,False,False,None,None,False
1,2025-01-01,11:20,BROOKLYN,11213,40.666580,-73.939674,"(40.66658, -73.939674)",ALBANY AVE,CARROLL ST,None,...,None,None,None,None,None,True,False,None,True,True
2,2025-01-03,11:35,MANHATTAN,10033,40.854490,-73.934620,"(40.85449, -73.93462)",W 187 ST,BENNETT AVE,None,...,Sedan,None,None,None,None,False,False,None,None,False
3,2025-01-01,20:23,BROOKLYN,11235,40.589985,-73.955670,"(40.589985, -73.95567)",AVENUE Y,E 14 ST,None,...,Sedan,None,None,None,None,True,False,None,True,True
4,2025-01-01,21:45,STATEN ISLAND,10306,40.570465,-74.109764,"(40.570465, -74.109764)",HYLAN BLVD,NEW DORP LN,None,...,Sedan,Sedan,None,None,None,True,False,None,True,True
5,2025-01-01,9:25,None,None,40.556152,-74.210450,"(40.556152, -74.21045)",WEST SHORE EXPRESSWAY RAMP,None,None,...,Station Wagon/Sport Utility Vehicle,None,None,None,None,True,False,None,True,True
6,2025-01-02,17:39,BRONX,10459,40.818798,-73.892120,"(40.818798, -73.89212)",BRUCKNER BLVD,BARRETTO ST,None,...,Sedan,Station Wagon/Sport Utility Vehicle,None,None,None,False,False,None,None,False
7,2025-01-02,6:50,QUEENS,11368,40.740757,-73.853110,"(40.740757, -73.85311)",None,None,55-20 VAN CLEEF ST,...,Sedan,None,None,None,None,False,False,None,None,False
8,2025-01-03,7:50,BROOKLYN,11201,40.688957,-73.981415,"(40.688957, -73.981415)",None,None,540 FULTON ST,...,Station Wagon/Sport Utility Vehicle,Bike,None,None,None,False,False,None,None,False
9,2025-01-03,13:22,MANHATTAN,10010,40.733242,-73.975050,"(40.733242, -73.97505)",None,None,601 E 20 ST,...,Sedan,None,None,None,None,False,False,None,None,False
